# 🚀 Clasificador de Textos Clásicos - Google Colab

Este notebook te permite entrenar el modelo usando la GPU gratuita de Google Colab.

**Tiempo estimado total:** 30-45 minutos (con GPU)

---

## ⚙️ Configuración Inicial

In [ ]:
# Verificar GPU disponible
!nvidia-smi

Si ves información de la GPU, ¡perfecto! Si no:
1. Ve a **Runtime → Change runtime type**
2. Selecciona **GPU** en Hardware accelerator
3. Guarda y vuelve a ejecutar la celda anterior

In [ ]:
# Clonar el repositorio (ajusta la URL)
!git clone https://github.com/TU_USUARIO/proyecto2.git
%cd proyecto2

In [ ]:
# Instalar dependencias
!pip install -q -r requirements.txt

---

## 🔐 Configurar Credenciales de MongoDB

**IMPORTANTE:** Usa Colab Secrets para no exponer tus credenciales.

1. Haz clic en el ícono de 🔑 (llave) en el panel izquierdo
2. Agrega dos secretos:
   - **Nombre:** `MONGO_URI` → **Valor:** tu URI de MongoDB Atlas
   - **Nombre:** `MONGO_DB_NAME` → **Valor:** `textos_clasicos`
3. Activa el acceso al notebook

In [ ]:
# Cargar credenciales desde Colab Secrets
from google.colab import userdata
import os

os.environ['MONGO_URI'] = userdata.get('MONGO_URI')
os.environ['MONGO_DB_NAME'] = userdata.get('MONGO_DB_NAME')

# Crear archivo .env para compatibilidad
with open('.env', 'w') as f:
    f.write(f"MONGO_URI={os.environ['MONGO_URI']}\n")
    f.write(f"MONGO_DB_NAME={os.environ['MONGO_DB_NAME']}\n")

print('Credenciales configuradas')

---

## 1️⃣ Verificar Conexión a MongoDB

In [ ]:
!python scripts/01_test_connection.py

---

## 2️⃣ Extracción de Datos (ETL)

Migra los datos de Excel a MongoDB.

In [ ]:
!python scripts/02_run_etl.py

**Resultado esperado:** 123 documentos insertados

---

## 3️⃣ Preprocesamiento y Balanceo

In [ ]:
!python scripts/03_preprocess.py

---

## 4️⃣ Entrenamiento del Modelo (GPU)

**⏱️ Tiempo estimado:** 15-30 minutos con GPU

In [ ]:
!python scripts/04_train.py

---

## 5️⃣ Evaluación del Modelo

In [ ]:
!python scripts/05_evaluate.py

### Visualizar Reportes

In [ ]:
from IPython.display import Image, display
import json

# Mostrar matriz de confusion
print('Matriz de Confusion:')
display(Image('reports/confusion_matrix.png'))

print('\nMetricas por Clase:')
display(Image('reports/metrics_by_class.png'))

# Mostrar reporte JSON
print('\nReporte Completo:')
with open('reports/evaluation_report.json', 'r') as f:
    report = json.load(f)
    print(json.dumps(report, indent=2))

---

## 6️⃣ Probar el Modelo (Inferencia)

In [ ]:
# Cargar el modelo
from src.model.inference import TextClassifier

classifier = TextClassifier()
classifier.load()

print('Modelo cargado')

In [ ]:
# Funcion para clasificar texto
def clasificar(texto):
    resultado = classifier.predict(texto, return_all_scores=True)
    
    print(f"\nTexto: {texto}")
    print(f"\nCategoria: {resultado['categoria_display']}")
    print(f"Confianza: {resultado['confianza_porcentaje']}")
    print(f"\nDescripcion: {resultado['descripcion']}")
    print(f"\nTodas las probabilidades:")
    for cat, prob in resultado['todas_probabilidades'].items():
        print(f"   - {cat}: {prob*100:.1f}%")

# Ejemplos de prueba
textos_prueba = [
    "La virtud es el camino hacia la excelencia del alma.",
    "El rey gobierna con sabiduria sobre su pueblo.",
    "Los dioses intervienen en los asuntos de los mortales."
]

for texto in textos_prueba:
    clasificar(texto)
    print("\n" + "="*60)

### 🧪 Prueba tu Propio Texto

In [ ]:
# Ingresa tu texto aqui
mi_texto = "Escribe aqui el texto que quieres clasificar"

clasificar(mi_texto)

---

## 💾 Descargar el Modelo Entrenado

Para usar el modelo localmente despues:

In [ ]:
# Comprimir el modelo
!zip -r modelo_entrenado.zip models/clasificador_textos/final/

# Descargar
from google.colab import files
files.download('modelo_entrenado.zip')

print('\nModelo descargado. Descomprimelo en tu carpeta models/ local.')

---

## 🎉 ¡Listo!

Has completado todo el pipeline en Google Colab. Ahora puedes:

1. **Descargar el modelo** y usarlo localmente
2. **Probar más textos** en la seccion de inferencia
3. **Ajustar hiperparametros** y reentrenar si es necesario
4. **Compartir los resultados** con tu equipo

### 📝 Notas Importantes:

- Este notebook usa la GPU gratuita de Colab (limite de ~12 horas)
- Los archivos se borran cuando cierras la sesion
- Descarga el modelo antes de cerrar
- Los datos permanecen en MongoDB Atlas